In [1]:
from azureml.core import Workspace, Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.experiment import Experiment
from azureml.pipeline.wrapper import PipelineRun, Module, dsl

### Prepare your workspace

In [2]:
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
InteractiveLoginAuthentication(tenant_id=tenant_id)
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

fundamental3
fundamental
eastasia
4f455bd0-f95a-4b7d-8d08-078611508e0b
dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy'])


### Choose a compute target

In [3]:
aml_compute_name = 'aml-compute'
try:
    aml_compute = AmlCompute(workspace, aml_compute_name)
    print("Found existing compute target: {}".format(aml_compute_name))
except:
    print("Creating new compute target: {}".format(aml_compute_name))

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=1, max_nodes=4)
    aml_compute = ComputeTarget.create(workspace, aml_compute_name, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target: aml-compute


### Load data

In [4]:
dataset_name = "THUCNews_For_Batch_Inference"
dataset_score  = workspace.datasets[dataset_name]

### Load module

In [5]:
fasttext_score_module_func = Module.from_yaml(workspace, 'fasttext_score/fasttext_score.spec.yaml')

### Load model

In [6]:
exp_name_list = [exp.name for exp in Experiment.list(workspace)]
exp_name_list

['fasttext_test',
 'sample-pipelines',
 'automobile',
 'fasttext_predict',
 'sample-pipelines2',
 'fasttext_with_two_training_process',
 'train-within-notebook',
 'train-on-local',
 'logging-api-test',
 'fasttext_with_one_training_process',
 'fasttext_train',
 'my_test',
 'split_data_txt',
 'compare_two_models',
 'yucongj-test',
 'fasttext_parallel_score',
 'parallel',
 'dir',
 'test0717',
 'test_0727',
 'test_0727_experiment',
 'localtest',
 'mpi_0729',
 'mpi_0729_experiment',
 'test',
 'para_0729',
 'para_0729_experiment',
 'basic_0721',
 'basic_0721_experiment',
 'deploy',
 'fasttext_training_process',
 'fasttext_batch_inference']

### Choose the experiment you want with its name

In [7]:
experiment_name = "fasttext_training_process"
experiment = Experiment(workspace, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
fasttext_training_process,fundamental3,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
# azureml.pipeline.core.run.PipelineRun
run = experiment.get_runs().__next__()
run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,ee7d355b-48e2-46a1-a803-f878ff7a549f,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Get a PipelineRun object

In [9]:
run_id = run.id
# azureml.pipeline.wrapper._pipeline_run.PipelineRun
pipeline_run = PipelineRun(experiment, run_id)
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,ee7d355b-48e2-46a1-a803-f878ff7a549f,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Visualize the pipeline

In [10]:
step_run_list = pipeline_run.PipelineRun_find_step_run(name='FastText Train')
for s in step_run_list:
    print(s,'\n\n')

StepRun(Experiment: fasttext_training_process,
Id: a0ffbff0-8961-4281-87b2-83459dcacc62,
Type: azureml.StepRun,
Status: Completed) 


StepRun(Experiment: fasttext_training_process,
Id: df736570-dd90-4eda-a6d6-048f7e30ccdb,
Type: azureml.StepRun,
Status: Completed) 




### Get a StepRun object

In [11]:
# step_run_id will be from the visualization result.
step_run_id = '530c79d5-3304-4cbe-92af-c45ce31fe98b'
step_run = pipeline_run.get_step_run(step_run_id)
step_run

Experiment,Id,Type,Status,Details Page,Docs Page
fasttext_training_process,530c79d5-3304-4cbe-92af-c45ce31fe98b,azureml.StepRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


### Use the trained model as the input of a new pipeline

In [12]:
# name will be from the visualization result.
# get_port() should supports three kinds of names: (1)the_better_model (2)The better model (3)The_better_model
port = step_run.get_port(name='The_better_model')
data_path = port.get_data_path()
model = Dataset.File.from_files(path=[data_path]).as_named_input('model_for_batch_inference')

### Construct the pipeline

In [13]:
@dsl.pipeline(name='batch inference', description='Batch Inference', default_compute_target=aml_compute.name)
def training_pipeline():
    fasttext_score = fasttext_score_module_func(
        texts_to_score=dataset_score,
        fasttext_model_dir=model
    )
    fasttext_score.runsettings.configure(node_count=2, process_count_per_node=2, mini_batch_size="64")

    return {**fasttext_score.outputs}

In [14]:
# pipeline
pipeline = training_pipeline()
# pipeline.save(experiment_name=experiment_name)

In [15]:
# validate
pipeline.validate()

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_4490afdd-92b8-4dad-8bbc-3616513a6461_widget', env_json='{"subscription…

{'result': 'validation passed', 'errors': []}

In [16]:
# pipeline_run
experiment_name = 'fasttext_batch_inference'
pipeline_run = pipeline.submit(experiment_name=experiment_name, regenerate_outputs=True)
pipeline_run.wait_for_completion()

Submitted PipelineRun 7667659f-fcca-4ab9-bf32-fa788c616c7c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_batch_inference/runs/7667659f-fcca-4ab9-bf32-fa788c616c7c?wsid=/subscriptions/4f455bd0-f95a-4b7d-8d08-078611508e0b/resourcegroups/fundamental/workspaces/fundamental3
PipelineRunId: 7667659f-fcca-4ab9-bf32-fa788c616c7c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_batch_inference/runs/7667659f-fcca-4ab9-bf32-fa788c616c7c?wsid=/subscriptions/4f455bd0-f95a-4b7d-8d08-078611508e0b/resourcegroups/fundamental/workspaces/fundamental3


<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_6417c5fd-e40a-4ac5-bbc3-67d76ee70753_widget', env_json='{}', graph_jso…

<RunStatus.completed: 'Completed'>